In [1]:
import pandas as pd
import numpy as np
from scipy import sparse

In [2]:
DAMPING_FACTOR = 0.15

# Leitura e organização dos dados "data"

In [3]:
data = pd.read_csv("soc-sign-bitcoinotc.csv", sep=',', names=["SOURCE", "TARGET", "RATING", "TIME"], index_col="RATING", usecols=["SOURCE", "TARGET", "RATING"])
data.sort_index(ascending=False, inplace=True)
data = data.ix[10:8]
data.sort_values(by=["TARGET", "SOURCE"], inplace=True)
data.reset_index(drop=True, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


# Matrizes para calculo do PageRank

In [4]:
sources = data["SOURCE"].tolist()
targets = data["TARGET"].tolist()
n = max( max(sources, key=int), max(targets, key=int)) + 1

transfer = list()
for source in sources:
        transfer.append(1/len(data[data.SOURCE == source]))

matrix_a = sparse.coo_matrix((transfer,( targets, sources)), shape=(n,n)).toarray()

In [5]:
matrix_b = np.full( (n, n), (DAMPING_FACTOR/n))

In [6]:
matrix_m = (( 1 - DAMPING_FACTOR) * matrix_a ) + ( DAMPING_FACTOR * matrix_b)

In [7]:
matrix_v = np.full( (n, 1), (1/n))

# Função para calculo do PageRank

In [8]:
count = 0
def pagerank(matrix_v):
    global count
    count +=1
    if((sum(abs((matrix_m.dot(matrix_v))-matrix_v))) > 0.001):
        return pagerank(matrix_m.dot(matrix_v))
    else:
        return matrix_m.dot(matrix_v)

# Criação do arquivo CSV com os PageRank em ordem decrescente, para visualização no Gephi.

In [9]:
result = [value[0] for value in pagerank(matrix_v).tolist()]
pagerank_csv = {'pagerank':result}
pd.DataFrame(pagerank_csv).sort_values(by=['pagerank'], ascending=False).to_csv("pagerank.csv", index_label="investor", columns=["pagerank"])

# Quantas iterações o PageRank precisou rodar até atingir convergência?

In [10]:
count

14

# Quais os 5 investidores mais importantes segundo o PageRank? Quais seus valores de PageRank?

In [11]:
pd.read_csv('pagerank.csv').head(5)

,investor,pagerank
0,144,0.000083
1,202,0.000070
2,361,0.000065
3,3996,0.000051
4,1,0.000037


# Como você poderia usar o PageRank caso você fosse um investidor em bitcoins?

In [12]:
# Dando preferencia as negociações com os investidores de melhor PageRank, devido os mesmos terem maior confiança da comunidade do Bitcoin.